In [13]:
import pandas as pd
import numpy as np
from numpy.random import rand
import matplotlib.pyplot as plt
from functools import reduce
from functools import wraps

In [2]:
from visualizacoes import display_vec, display_matrix

# MultiLayer Perceptron

<img src='https://www.learnopencv.com/wp-content/uploads/2017/10/mlp-diagram.jpg' width='600px'>

Podemos escrever a trasferência (somatória) do neurônio $k$ da camada $l$ de $z^{l}_{k}$ apenas adaptando um pouco a equação do perceptron simples adicionando a notação de índice das camadas de neuronios e substituindo $x$ por $a$ uma vez que o input de um neurônio da camada $l$ são as ativações $a$ dos neurônios da camada $l-1$:

$$ \normalsize z^{l}_{k} = \sum_{i=1}^{m} w^{l}_i a^{l - 1}_i + b^{l}_k $$ 

E a ativação do neurônio $z^{l}_{k}$ é dado por:

$$ \normalsize a^{l}_{k} = \sigma(z^{l}_{k}) $$


Quanto à camada de saída de uma rede neural, temos algumas coisas a se pensar. Podemos ter duas situações: classificação __binária__ (duas classes, como 0 e 1) ou __multiclasses__ (como, 1,2,3 ou mesmo, "gato", "cachorro", "pessoa", etc). Dependendo do caso, podemos optar por apenas um neurônio de saída (que geralmente terá ativação sigmoide/logística ou step), ou, caso tenhamos mais do que duas classes, podemos trabalhar com a função __softmax__, que é uma adaptação vetorial da função sigmoide que comporta múltiplas classes.

Supondo que ao invés de ter um neurônio de saída, tenhamos $K$ neurônios, um para cada uma das $k$ classes possíveis. Seja $\vec{a}$ um vetor contendo a ativação dos $k$ neurônios de saída, podemos utilizar a função softmax da seguinte forma:

(1) $$ \normalsize \vec{a} = \begin{bmatrix}a_1\\ a_2\\ \vdots\\ a_k\end{bmatrix} $$

a função softmax é dada por:

(2) $$ \normalsize \text{softmax}(\vec{a}) = \frac{e^{a_i}}{\sum_{j=1}^{k} e^{a_j}} $$

Como resultado, teremos um vetor de 0 a 1 que pode ser interpretado como a __probabilidade__ para cada classe:

(3) $$ \normalsize \text{softmax}(\vec{a}) = \begin{bmatrix}P(a_1|\vec{a})\\ P(a_2|\vec{a})\\ \vdots\\ P(a_k|\vec{a})\end{bmatrix} $$

Sendo assim, a classificação é dada pela classe com maior probabilidade:

(4) $$ \text{argmax}(\text{softmax}(\vec{a})) $$

Vamos ver um exemplo com um vetor $\vec{a}$  aleatório:

In [89]:
a = np.array([1.3, 2.2, 4.5])

In [90]:
display_vec(a, '\\vec{a}')

<br>$\vec{a} = \begin{bmatrix}1.30\\2.20\\4.50\end{bmatrix}$<br>

In [91]:
def softmax(a):
    somatoria = sum([e(a_i) for a_i in a])
    return np.array([e(a_i)/somatoria for a_i in a])

In [92]:
probs_vec = softmax(a)

In [93]:
display_vec(probs_vec, '\\text{softmax}(\\vec{a})')

<br>$\text{softmax}(\vec{a}) = \begin{bmatrix}0.04\\0.09\\0.88\end{bmatrix}$<br>

In [94]:
probs_vec.argmax()

2

In [95]:
probs_vec.sum()

1.0

In [96]:
import neuralnet_functions as funcs

In [97]:
funcs.ReLU.__doc__

'\n    Rectified Linear Unit\n    x: escalar, mas a função suporta vetores através do decorador vectorize_func\n    '

In [98]:
func = softmax
print(func.__name__)
print(func.__doc__)

softmax
None


In [99]:
class LinearLayer():
    
    def __init__(self, input_shape, n_neurons, activation):
        self.n_inputs = input_shape
        self.n_outputs = n_neurons
        self.weights = np.random.rand(input_shape, n_neurons)
        self.bias = np.random.rand(n_neurons)
        self.erro = None
        self.z = None
        self.a = None
        self.func = activation
        self.n_params = reduce((lambda x, y: x * y), self.weights.shape) + n_neurons
    
    def transfer(self, X):
        self.z = np.dot(X, self.weights) + self.bias
        self.a = self.func(self.z)

#### Construindo uma arquitetura de MLP

A imagem abaixo mostra a arquitetura que vamos construir. Os índices $I$, $H1$, $H2$ e $O$ representam os layers da rede neural, enquanto que os termos $W$ representam as matrizes de pesos entre as camadas. Os termos $n$, $m$, $k$ representam os índices da contagem de neurônios por layer. Essas notações serão importantes para que possamos entender bem durante a construção da rede, quais valores representam exatamente quais termos dentro da arquitetura da rede neural.


<img src='imgs/MLP_architeture_overall.png' width='500px'>

### Feedforward

Abaixo exibimos o primeiro vetor da matriz $X$ contendo as variáveis independentes. Como alimentaremos a rede neural com vetores em $R^4$, precisamos construir uma camada de entrada (Input layer) com 4 neurônios - um para cada variável independente $x_n$. Repare abaixo que a matriz $X$ está ordenada como `n_samples` x `n_features` , ou seja, cada registro é uma linha, cada coluna é uma variável. Um ponto importante de ser mencionado, é que essas ordenações afetam a maneira com que armazenamos os pesos da rede neural. Algumas implementações ordenam $X$ como `n_samples` x `n_features` e a matriz de pesos como `n_outputs`x `n_inputs`, e outras, ao contrário. Aqui nessa demonstração, iremos manter a matriz $X$ na ordenação atual, e armazenaremos os coeficientes de pesos como matrizes `n_inputs` x `n_outputs`. Por exemplo, considere a matriz de pesos $W^{2}$. Ela terá formato 5 x 2:

$$ \normalsize W^{2} = \begin{bmatrix} w_{11} & w_{12}\\  w_{21} & w_{22} \\  w_{31} & w_{32} \\  w_{41} & w_{42}\\  w_{51} & w_{52}\end{bmatrix} $$


A matriz $W^2$, que armazena os pesos entre as camadas $H1$ e $H2$ possuí dimensões $m$ x $k$. Como mencionado, estamos armazenando os pesos como `n_inputs` x `n_outputs`. Cada linha representa os pesos que saem de um espectivo neurônio $k$ da camada $H1$ e cada coluna representa as conexões que chegam para cada neurônio da camada $H2$ conforme a ilustração abaixo:

<img src='imgs/MLP_architeture_W2.png'>

In [100]:
display_matrix(X.values, n_rows=1, label='X')

<br>$X = \begin{bmatrix}1&0&1&0.58\end{bmatrix}$<br><br>

Dimensões da matriz: (2001 x 4)



Primeiro hidden Layer, com 8 neurônios e ativação sigmoide:

In [103]:
hiddenLayer1 = LinearLayer(input_shape=4, n_neurons=5, activation=funcs.sigmoide)

In [104]:
display_matrix(hiddenLayer1.weights, label='W^1')
display_vec(hiddenLayer1.bias, 'b^1')

<br>$W^1 = \begin{bmatrix}0.65&0.29&0.15&0.80&0.89\\0.73&0.09&0.65&0.41&0.04\\0.62&0.64&0.12&0.31&0.58\\0.77&0.66&0.32&0.14&0.20\end{bmatrix}$<br><br>

Dimensões da matriz: (4 x 5)



<br>$b^1 = \begin{bmatrix}0.25\\0.12\\0.44\\0.27\\0.23\end{bmatrix}$<br>

Por exemplo, para o primeiro input $x_1$, a transferência desse input no primeiro neurônio será:

$$ \normalsize z^{1}_1 =  w_{11} x_{11} + w_{21} x_{12} +  w_{31} x_{13} +  w_{41} x_{14} + b_1 $$

e a ativação deste neurônio:

$$ \normalsize a^{1}_1 = \sigma(z^{1}_1)$$

$$ \normalsize a^{1}_1 = \frac{1}{1 + e^{-(z^{1}_1)}} $$

In [105]:
hiddenLayer1.transfer(X)

Transferência $z$ dos 8 neurônios 

In [106]:
display_matrix(hiddenLayer1.z, n_rows=1, label='z^1')

<br>$z^1 = \begin{bmatrix}1.97&1.43&0.90&1.46&1.83\end{bmatrix}$<br><br>

Dimensões da matriz: (2001 x 5)



Ativação dos 8 neurônios para $x_1$

In [107]:
display_matrix(hiddenLayer1.a, n_rows=1, label='a^1')

<br>$a^1 = \begin{bmatrix}0.88&0.81&0.71&0.81&0.86\end{bmatrix}$<br><br>

Dimensões da matriz: (2001 x 5)



Criando uma segunda hidden layer com 4 neurônios:

O input para cada neurônio deste layer será a quantidade de neurônios na camada anterior, e o formato da matriz de pesos que conectam ambos os layers é dado por: número de neurônios na camada $l-1$ x número de neurônios na camada $l$. No nosso caso, 8x4

In [109]:
hiddenLayer2 = LinearLayer(5, 2, funcs.sigmoide)

In [110]:
display_matrix(hiddenLayer2.weights, label='W^2')
display_vec(hiddenLayer2.bias, 'b^2')

<br>$W^2 = \begin{bmatrix}0.37&0.75\\0.01&0.80\\0.72&0.02\\0.72&0.19\\0.55&0.52\end{bmatrix}$<br><br>

Dimensões da matriz: (5 x 2)



<br>$b^2 = \begin{bmatrix}0.03\\0.49\end{bmatrix}$<br>

In [111]:
hiddenLayer2.transfer(hiddenLayer1.a)

Ativação dos 4 neurônios para os 5 primeiros inputs

In [112]:
display_matrix(hiddenLayer2.a, n_rows=5, label='a^2')

<br>$a^2 = \begin{bmatrix}0.87&0.92\\0.87&0.92\\0.87&0.92\\0.87&0.92\\0.85&0.89\end{bmatrix}$<br><br>

Dimensões da matriz: (2001 x 2)



Criando um output layer com um neurônio e ativação sigmoide

In [113]:
outputLayer = LinearLayer(2, 1, funcs.sigmoide)

In [114]:
display_matrix(outputLayer.weights, label='W^L')
display_vec(outputLayer.bias, label='b^L')

<br>$W^L = \begin{bmatrix}0.41\\0.87\end{bmatrix}$<br><br>

Dimensões da matriz: (2 x 1)



<br>$b^L = \begin{bmatrix}0.29\end{bmatrix}$<br>

In [115]:
outputLayer.transfer(hiddenLayer2.a)

In [116]:
display_matrix(outputLayer.a, n_rows=5, label='a^L')

<br>$a^L = \begin{bmatrix}0.81\\0.81\\0.81\\0.81\\0.81\end{bmatrix}$<br><br>

Dimensões da matriz: (2001 x 1)



 ### Backpropagation
 
 #### Sinal do erro
 
 Sinal de erro de um neurônio $j$ da camada $l$ é basicamente quantificar o quanto a função de custo variou (para mais ou para menos) conforme a transferência $z$ do neurônio aumentou ou diminuiu:
 
 $$ \normalsize \delta^{L}_{j} = \frac{\partial C}{\partial z^{L}_{j}} $$
 
 Onde $z^{l}_{j}$ é a transferência do neurônio, e $C$ é a função de custo da rede neural. 
 
 Acontece que quando a transferência $z$ de um neurônio varia, a ativação $a$ deste neurônio também varia, uma vez que a ativação do neurônio é dada por:
 
 $$ \normalsize a^{L}_{j} = \sigma(z^{L}_{j}) $$
 
 Sendo assim, a equação (1) é expandida através da lei da cadeia e se transforma em:
 
 $$ \normalsize \delta^{L}_{j} = \frac{\partial C}{\partial a^{L}_{j}} \frac{\partial a^{L}_{j}}{\partial z^{L}_{j}} $$
 
 $$ \normalsize \delta^{L}_{j} = \frac{\partial C}{\partial a^{L}_{j}} \sigma'(z^{L}_{j}) $$
 
 Essa equação pode ser interpretada como o quanto o erro varia dado a ativação do neurônio, e essa quantidade sendo multiplicada por quanto a ativação varia conforme a transferência do neurônio. Supondo que o erro $C$ cresça bastante para a ativação $a^{l}_{j}$, isso nos daria uma quantidade positiva para o primeiro termo. Acontece, que precisamos ponderar essa variação levando em consideração o quanto a ativação do neurônio altera para o valor de transferência $z$, e é por isso que o termo acima é multiplicado pelo segundo termo.

Supondo que a função de erro $C$ para o vetor de ativações $A$ e o vetor de valores resposta $Y$, seja o custo de erro quadrático, dado pela fórmula:

$$ \normalsize C(w,b) = \frac{1}{2n} \sum_{j=0}^{J} \parallel y - a^{L} \parallel^2 $$

Onde, os argumentos desta função são justamente os pesos e bias da rede neural $W$ e $b$, que levarão ao vetor de ativações $a$ na camada de saída, que serão comparados com os valores originais $y$.

Sendo assim, para um único $x_k$, o custo da sua previsão dado a ativação dos neurônios será:

$$ \normalsize C(w,b) = \frac{1}{2} \parallel y - a^{L} \parallel^2$$  


In [40]:
C = lambda y,a: (y - a)**2 / 2

In [41]:
C(1, 0.91)

0.004049999999999997

In [43]:
(1 - 0.91)**2

0.008099999999999994

In [118]:
display_vec(Y[:5], label='Y')

<br>$Y = \begin{bmatrix}0\\0\\0\\0\\0\end{bmatrix}$<br>

In [119]:
custo_por_input = np.array([C(y, a) for y, a in zip(Y, outputLayer.a)])

Aplicando a função de custo ás saídas da rede neural e suas respectivas respostas esperadas (y)

In [120]:
display_matrix(custo_por_input, n_rows=5, label='C(w,b)')

<br>$C(w,b) = \begin{bmatrix}0.33\\0.33\\0.33\\0.33\\0.32\end{bmatrix}$<br><br>

Dimensões da matriz: (2001 x 1)





Já a __variação__ do custo em respeito a ativação do neurônio pode ser encontrada usando a regra da potência em derivação:

$$ \normalsize f(x) = x^n \,\, \therefore \,\, \frac{d}{dx} x^n = nx^{n-1}$$

Sendo assim, voltando para o cálculo do gradiente de erro no neurônio de saída, temos então que o primeiro termo da equação pode ser computado como:

$$ \normalsize \frac{\partial C}{\partial a^{l}_{j}} = \frac{2}{2}(y_j - a^{L}_j)^{2-1}$$

$$ \normalsize \frac{\partial C}{\partial a^{l}_{j}} = (y_j - a^{L}_j)$$

Já para o segundo termo na equação do gradiente da camada de saída, precisamos encontrar a taxa de variação da função de ativação com respeito a transferência do neurônio (representado por $z$). Supondo que usamos a função sigmoide:

$$ \normalsize \sigma(z) = \frac{1}{1 + e^{-z}} $$

Sua função derivada será:

$$ \normalsize \sigma'(z) = \sigma(z)(1 - \sigma(z)) $$

Portanto, o erro $\delta^{L}_{j}$ para uma rede com função de ativação sigmoide e função de custo quadrática é dado por:

$$ \normalsize \delta^{L} = (a^{L} - y) \odot \sigma(z)(1 - \sigma(z)) $$

Essa equação, sendo generalizada para outras funções de custo e ativação, transforma-se na equação abaixo, que é bastante vista em documentações, papers e artigos sobre o algoritmo backpropagation:

$$ \normalsize \delta^{L} = \nabla_a C \odot \sigma'(z) $$

Onde $\nabla_a C$ costuma representar um vetor contendo as derivadas parciais $ \frac{\partial C}{\partial a^{l}_{j}}$, $\odot$ representa uma operação de multiplicação <em>element-wise</em> entre vetores e $\sigma'(z)$ representa a função derivada da função de ativação. No nosso caso, temos dois neurônios na camada de saída, ambos com ativação sigmoide. O vetor de erros da camada de output será então:

(1) $$ \normalsize \delta^{L} = \begin{bmatrix}(a^{L}_1 - y_1)\\(a^{L}_2 - y_2)\end{bmatrix} \odot  \begin{bmatrix}\sigma'(z^{L}_1)\\\sigma'(z^{L}_2)\end{bmatrix}$$

(2) $$ \normalsize \delta^{L} = \begin{bmatrix}(a^{L}_1 - y_1)\,\sigma'(z^{L}_1)\\(a^{L}_2 - y_2)\,\sigma'(z^{L}_2)\end{bmatrix}$$

In [121]:
derivada_sigmoide = lambda z: funcs.sigmoide(z)*(1 - funcs.sigmoide(z))
derivadas_parciais =  lambda a, y: a - y

In [122]:
nabla_a_C = np.array([derivadas_parciais(a, y) for a, y in zip(outputLayer.a, Y)]) 

In [123]:
display_matrix(nabla_a_C.T, n_rows=2, n_cols=1, label='\\nabla_a C')

<br>$\nabla_a C = \begin{bmatrix}0.81\end{bmatrix}$<br><br>

Dimensões da matriz: (1 x 2001)



In [124]:
deriv_sigma_z = np.array([derivada_sigmoide(z) for z in outputLayer.z])

In [125]:
display_matrix(deriv_sigma_z.T, n_rows=2, n_cols=1, label="\sigma'(z)")

<br>$\sigma'(z) = \begin{bmatrix}0.15\end{bmatrix}$<br><br>

Dimensões da matriz: (1 x 2001)



In [126]:
outputLayer.error = (nabla_a_C * deriv_sigma_z).T

In [127]:
outputLayer.error.shape

(1, 2001)

In [128]:
display_matrix(outputLayer.error, n_rows=2, n_cols=1, label='\\delta^{L}')

<br>$\delta^{L} = \begin{bmatrix}0.12\end{bmatrix}$<br><br>

Dimensões da matriz: (1 x 2001)



#### Gradiente da função de erro em respeito aos pesos

O objetivo do algoritmo de backpropagation, como mencionado, é encontrar a combinação de pesos da rede neural que em conjunto retornam o menor erro. O gradiente da função de custo em relação ao peso $w_{ij}$ da camada $l$ da rede neural é dado por:

$$ \normalsize \frac{\partial C}{\partial w^{L}_{ij}} $$

Através da __regra da cadeia__, podemos expandir para:


$$ \normalsize \frac{\partial C}{\partial w^{L}_{ij}} = \frac{\partial C}{\partial z^{L}_{j}} \frac{\partial z^{L}_{j}}{\partial w^{L}_{ij}}$$

Já sabemos que $\frac{\partial C}{\partial z^{L}_{j}}$ é o sinal do erro:

 $$ \normalsize \delta^{L}_{j} = \frac{\partial C}{\partial z^{L}_{j}} $$
 
 Já a parcial $\frac{\partial z^{L}_{j}}{\partial w^{L}_{ij}}$ é a ativação do neurônio de entrada da camada:
 
 $$ \normalsize \frac{\partial z^{L}_{j}}{\partial w^{L}_{ij}} = a^{l-1}_i $$
 
 Finalmente, podemos reescrever a equação do gradiente do custo em relação ao peso $w_{ij}$ como:
 
 $$ \normalsize \frac{\partial C}{\partial w^{L}_{ij}} = \delta^{L}_{j}\,a^{l-1}_i  $$

In [129]:
display_matrix(hiddenLayer2.a, n_rows=5, label='a^2')

<br>$a^2 = \begin{bmatrix}0.87&0.92\\0.87&0.92\\0.87&0.92\\0.87&0.92\\0.85&0.89\end{bmatrix}$<br><br>

Dimensões da matriz: (2001 x 2)



In [130]:
gradientes_output = np.dot(hiddenLayer2.a.T, outputLayer.error.T)

In [131]:
display_matrix(gradientes_output)

<br>$\begin{bmatrix}81.97\\86.38\end{bmatrix}$<br><br>

Dimensões da matriz: (2 x 1)



#### Propagando o erro nas camadas ocultas

Conseguimos chegar ao vetor de gradientes da camada de saída. 

Supondo que a última camada oculta da nossa rede seja a camada $l$. O que faremos é multiplicar o erro calculado nos neurônios da camada de saída $L$ pelos pesos que conectam a camada $l$ a camada $L$ (volte na __visualização da matriz de pesos__ da camada de output), que como vimos, é uma matriz 4x2. Cada um desses 4 valores serão então multiplicados pela função derivada da sigmoide para cada um dos valores dos neurônios da camada $l$. O erro portanto, do neurônio $j$ da camada $l$ é dado por: 

$$ \Large \delta^{l}_j = (\sum_{j=1}^{k} w^{l + 1}_j \delta^{l+1}_j)\, \sigma'(z^l_j) $$

Onde $w^{l + 1}$ é a matriz de pesos da camada $l + 1$ e $\delta^{l+1}$ é o vetor de sinais de erro da camada $l + 1$. Levando em consideração que armazenamos o erro dos neurônios de saída em um vetor, e os pesos estão armazenados em uma matriz $W$, podemos reescrever a equação acima como:

$$ \Large \delta^{l} = W^{l + 1} \delta^{l+1} \odot \sigma'(z^l) $$

Voltando ao nosso caso, queremos calcular os gradientes de erro da penultima camada da rede, a `hiddenLayer2`. Precisamos calcular então ambos os termos da equação acima novamente. O primeiro é a multiplicação da matriz de pesos da camada de saída $L$ pelo vetor de erros dos dois neurônios da camada $L$, e finalmente multiplicar esses valores pelas respectivas derivadas das ativações dos neurônios da `hiddenLayer2`, representadas aqui como a camada $l$:

$$ \normalsize \delta^{l} = \begin{bmatrix} W^L_1\delta^{L} \\ W^L_2\delta^{L}\\ W^L_3\delta^{L}\\ W^L_4\delta^{L}\end{bmatrix} \odot  \begin{bmatrix}\sigma'(z^{l}_1)\\\sigma'(z^{l}_2)\\ \sigma'(z^{l}_3)\\\sigma'(z^{l}_4)\end{bmatrix} $$

Com base nisso, vamos calcular o gradiente de erro para o `hiddenLayer2`. Primeiro calculamos o vetor contendo $W^L \delta^L$

In [132]:
W_L_Delta = np.dot(outputLayer.weights, outputLayer.error)

In [133]:
display_matrix(W_L_Delta, n_cols=1, label='W^L \delta^L')

<br>$W^L \delta^L = \begin{bmatrix}0.05\\0.11\end{bmatrix}$<br><br>

Dimensões da matriz: (2 x 2001)



In [134]:
deriv_hiddenlayer2 = np.array([derivada_sigmoide(z) for z in hiddenLayer2.z])

In [135]:
display_matrix(deriv_hiddenlayer2.T, n_cols=1, label="\sigma'(z)")

<br>$\sigma'(z) = \begin{bmatrix}0.11\\0.08\end{bmatrix}$<br><br>

Dimensões da matriz: (2 x 2001)



In [136]:
hiddenLayer2.error = W_L_Delta * deriv_hiddenlayer2.T

In [137]:
display_matrix(hiddenLayer2.error, n_cols=1, label='\\delta^{l}')

<br>$\delta^{l} = \begin{bmatrix}0.01\\0.01\end{bmatrix}$<br><br>

Dimensões da matriz: (2 x 2001)



Propagando o erro para a primeira camada oculta

In [138]:
W_L_Delta_1 = np.dot(hiddenLayer2.weights, hiddenLayer2.error)

In [139]:
display_matrix(W_L_Delta_1, n_cols=1, label='W^L \delta^L')

<br>$W^L \delta^L = \begin{bmatrix}0.01\\0.01\\0.00\\0.01\\0.01\end{bmatrix}$<br><br>

Dimensões da matriz: (5 x 2001)



In [140]:
deriv_hiddenlayer1 = np.array([derivada_sigmoide(z) for z in hiddenLayer1.z])

In [141]:
display_matrix(deriv_hiddenlayer1.T, n_cols=1, label="\sigma'(z)")

<br>$\sigma'(z) = \begin{bmatrix}0.11\\0.16\\0.21\\0.15\\0.12\end{bmatrix}$<br><br>

Dimensões da matriz: (5 x 2001)



In [142]:
hiddenLayer1.error = W_L_Delta_1 * deriv_hiddenlayer1.T

In [143]:
display_matrix(hiddenLayer1.error, n_cols=1, label='\\delta^{l}')

<br>$\delta^{l} = \begin{bmatrix}0.00\\0.00\\0.00\\0.00\\0.00\end{bmatrix}$<br><br>

Dimensões da matriz: (5 x 2001)



Gradiente da função de custo em relação aos pesos:

### Codificando um MLP

In [ ]:
class LinearLayer():
    
    def __init__(self, input_shape, n_neurons, activation):
        self.n_inputs = input_shape
        self.n_outputs = n_neurons
        self.weights = np.random.rand(input_shape, n_neurons)
        self.bias = np.random.rand(n_neurons)
        self.erro = np.zeros(n_neurons)
        self.z = None
        self.a = None
        self.func = activation
        self.n_params = reduce((lambda x, y: x * y), self.weights.shape) + n_neurons
    
    def transfer(self, X):
        self.z = np.dot(X, self.weights) + self.bias
        self.a = self.func(self.z)


class MultiLayerPerceptron():
    
    def __init__(self, n_inputs):
        self.n_inputs = n_inputs
        self.network = []
        self.trainable_params = 0
        self.n_layers = 0
        self.n_neurons = 0
        self.coefs = None
        self.intercepts = None
            
        
    def info(self):
        print('Número de neurônios: %d' % self.n_neurons)
        print('Número de layers: %d' % self.n_layers)
        print('Número de parâmetros treináveis: %d' % self.trainable_params)
        print('\nExibindo estrutura dos layers: \n')
        for i, vals in enumerate(self.network):
            print('Layer %d:' % i, '{}, '.format(vals.weights.shape), ' Parâmetros: %d' % self.network[i].n_params)

            
    def update_coefs(self, coefs, bias):
        for l in range(self.n_layers):
            self.network[l].weights = coefs[l]
            self.network[l].bias = bias[l]
            
        self.coefs = np.array([self.network[i].weights for i in range(self.n_neurons)])
        self.intercepts = np.array([self.network[i].bias for i in range(self.n_neurons)])            
            
            
    def add_layer(self, n_neurons, activation):
        if len(self.network) == 0:
            # primeiro layer da rede
            layer_obj = LinearLayer(self.n_inputs, n_neurons, activation)
        else:
            layer_input = self.network[-1].n_outputs
            layer_obj = LinearLayer(layer_input, n_neurons, activation)
            
        self.network.append(layer_obj)
        self.trainable_params += layer_obj.n_params
        self.n_layers = len(self.network)
        self.n_neurons += 1
        self.coefs = np.array([self.network[i].weights for i in range(self.n_neurons)])
        self.intercepts = np.array([self.network[i].bias for i in range(self.n_neurons)])
    
    
    def make_decision_function(self):
        thresholds = {'sigmoide':.5, 'tahn':0, 'sign':1}
        output_activation = self.network[-1].func.__name__
        
        if output_activation in thresholds.keys():
            return lambda a: 1 if a >= thresholds[output_activation] else 0
        
        elif output_activation == 'linear':
            return lambda a: a
        
        elif output_activation == 'softmax':
            return np.argmax
        
        else:
            raise 'Ativação da camada de saída não reconhecida'        
        

    def predict(self, X):
        for l in range(len(self.network)):
            print(self.network[l].weights)
            if l == 0:
                # se for input
                self.network[l].transfer(X)
            else:
                # se for output
                self.network[l].transfer(self.network[l - 1].a)
                
        A = self.network[-1].a
        decision_function = self.make_decision_function()
        return list(map(decision_function, A))

In [ ]:
def build_model(n_inputs, hiddenLayers=(6,), activation=funcs.sigmoide, n_classes=2, output_activation=funcs.sigmoide):
    # Instancia o modelo
    modelo = MultiLayerPerceptron(n_inputs = n_inputs)
    # constrói as camadas ocultas
    for layer in hiddenLayers:
        modelo.add_layer(layer, activation)
    # define camada de output
    if n_classes == 2:
        modelo.add_layer(1, output_activation)
    else:
        modelo.add_layer(n_classes, output_activation)
    # retorna objeto
    return modelo

In [ ]:
MLP = MultiLayerPerceptron(n_inputs=6)

In [ ]:
MLP.add_layer(4, funcs.sigmoide)

In [ ]:
MLP.add_layer(1, funcs.sigmoide)

In [ ]:
MLP.info()

In [ ]:
previsoes = MLP.predict(X)

In [ ]:
accuracy_score(Y, previsoes)

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(4), activation='logistic', solver='sgd')

In [ ]:
mlp.fit(X, Y)

In [ ]:
np.array(mlp.coefs_)

In [ ]:
mlp.intercepts_

In [ ]:
accuracy_score(Y, mlp.predict(X))

In [ ]:
new_coefs = np.array(mlp.coefs_)

In [ ]:
new_bias = np.array(mlp.intercepts_)

In [ ]:
MLP.update_coefs(new_coefs, new_bias)

In [ ]:
MLP.coefs

In [ ]:
previsoes = MLP.predict(X)

In [ ]:
accuracy_score(Y, previsoes)

In [ ]:
def derivative(func, x, dx=1e-6):
    return (func(x + dx) - func(x)) / dx

def funcaoDerivada(X, func, dx=1e-6):
    return [derivative(func, x, dx) for x in X]

def plotFuncDeriv(X, func):
    plt.plot(X, funcaoDerivada(X, func))
    plt.plot(X, list(map(func,X)))

In [ ]:
plotFuncDeriv(np.linspace(-10,10,100), funcs.sigmoide)

In [ ]:
plotFuncDeriv(np.linspace(-10,10,100), funcs.tanh)

In [ ]:
plotFuncDeriv(np.linspace(-10,10,100), funcs.ReLU)

In [ ]:
def QuadraticCost(Y, A):
    diff = (Y - A)**2
    return diff/2

In [ ]:
clf = build_model(n_inputs=6, hiddenLayers=(4,))

In [ ]:
clf.info()

In [ ]:
A = clf.predict(X)

In [ ]:
display_vec(A[:5])

In [ ]:
display_vec(Y[:5])

In [ ]:
erro_output = QuadraticCost(Y, A)

In [ ]:
erro_output[:5]